In [1]:
from synthetix_v3.base_perps import BasePerps
import polars as pl

In [2]:
base_perps = BasePerps()

In [3]:
settled_orders_df = base_perps.get_settled_orders()

In [9]:
polars_df = pl.from_pandas(settled_orders_df)

In [17]:
polars_subset_df = polars_df.select([
    pl.col('orderSettleds_timestamp'),
    pl.col('markets_marketSymbol'),
    # pl.col('orderSettleds_accountId'),
    pl.col('orderSettleds_fillPrice'),
    pl.col('orderSettleds_accruedFunding'),
    pl.col('orderSettleds_sizeDelta'),
    pl.col('orderSettleds_newSize'),
    pl.col('orderSettleds_totalFees'),
])

In [18]:
polars_subset_df.head(5)

orderSettleds_timestamp,markets_marketSymbol,orderSettleds_fillPrice,orderSettleds_accruedFunding,orderSettleds_sizeDelta,orderSettleds_newSize,orderSettleds_totalFees
i64,str,f64,f64,f64,f64,f64
1696391442,"""ETH""",1.6364e21,0.0,3.0426e16,3.0426e16,9.9580e15
1696391484,"""ETH""",1.6354e21,-3.1340e15,-4.7323e20,-4.7319e20,3.8696e20
1696553232,"""ETH""",1.6097e21,-1.0875e19,4.7319e20,0.0,1.5234e20
1696553262,"""ETH""",1.6109e21,-5.5683e15,1.5369e21,1.5369e21,4.9517e20
1696373616,"""ETH""",1.6523e21,0.0,4.1995e19,4.1995e19,3.4694e19


In [15]:
preprocessed_df = (polars_subset_df.with_columns([
    pl.from_epoch("orderSettleds_timestamp").alias("datetime"),
    (pl.col('orderSettleds_fillPrice') / 10 ** 18),
    (pl.col('orderSettleds_sizeDelta') / 10 ** 18),
    (pl.col('orderSettleds_accruedFunding') / 10 ** 18),
    (pl.col('orderSettleds_newSize') / 10 ** 18),
    (pl.col('orderSettleds_totalFees') / 10 ** 18),
])
.with_columns([
    (pl.col('orderSettleds_sizeDelta') * pl.col('orderSettleds_fillPrice')).alias('size_usd')
]))

In [20]:
preprocessed_df.filter(pl.col('markets_marketSymbol') == 'ETH').sort(by='datetime', descending=True)

orderSettleds_timestamp,markets_marketSymbol,orderSettleds_fillPrice,orderSettleds_accruedFunding,orderSettleds_sizeDelta,orderSettleds_newSize,orderSettleds_totalFees,datetime,size_usd
i64,str,f64,f64,f64,f64,f64,datetime[μs],f64
1696623652,"""ETH""",1646.340635,-0.095376,1913.4486,1913.4486,1011.547296,2023-10-06 20:20:52,3.1502e6
1696622800,"""ETH""",1643.103165,-1.683025,1831.1418,0.0,601.750977,2023-10-06 20:06:40,3.0088e6
1696619616,"""ETH""",1646.639647,-1.530655,-4232.76,-4232.76,2862.18444,2023-10-06 19:13:36,-6.9698e6
1696615874,"""ETH""",1649.48573,-0.017325,-904.9654,-904.9654,298.545503,2023-10-06 18:11:14,-1.4927e6
1696615732,"""ETH""",1651.746866,-0.696283,-2231.07,0.0,737.032576,2023-10-06 18:08:52,-3.6852e6
1696613908,"""ETH""",1655.409064,-0.293994,-151.5943,-397.5502,50.190116,2023-10-06 17:38:28,-250950.578219
1696610692,"""ETH""",1650.027905,-6.676232,-91.3281,-245.9559,30.138783,2023-10-06 16:44:52,-150693.913544
1696607002,"""ETH""",1659.161663,0.068805,-3083.2775,0.0,1023.131165,2023-10-06 15:43:22,-5.1157e6
1696602668,"""ETH""",1643.362737,0.001279,-1831.1418,-1831.1418,601.84604,2023-10-06 14:31:08,-3.0092e6


In [22]:
preprocessed_df.groupby('markets_marketSymbol').agg([pl.col('size_usd').abs().sum()]).sort(by='size_usd', descending=True)

/tmp/ipykernel_25720/3641139651.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  preprocessed_df.groupby('markets_marketSymbol').agg([pl.col('size_usd').abs().sum()]).sort(by='size_usd', descending=True)


markets_marketSymbol,size_usd
str,f64
"""BTC""",5.0966e8
"""ETH""",4.2216e8
"""OP""",9.1788e7
"""LINK""",5.2930e7
"""SNX""",2.7890e7


In [24]:
preprocessed_df.groupby('markets_marketSymbol').agg([pl.col('size_usd').abs().sum()]).sum()['size_usd'].item()

/tmp/ipykernel_25720/2835360431.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  preprocessed_df.groupby('markets_marketSymbol').agg([pl.col('size_usd').abs().sum()]).sum()['size_usd'].item()


1104432352.8035989